In [2]:
import tensorflow as tf
from tensorflow import keras

# TF Functions and Concrete Functions

In [3]:
@tf.function
def tf_cube(x):
    return x**3

In [4]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))

2022-04-20 18:42:16.072207: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-04-20 18:42:19.468553: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-20 18:42:19.468576: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benson516-P1G2): /proc/driver/nvidia/version does not exist
2022-04-20 18:42:19.468897: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
concrete_function

<ConcreteFunction tf_cube(x) at 0x7F292C51EA90>

In [6]:
concrete_function(tf.constant(2.0))

2022-04-20 18:42:53.305402: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-20 18:42:53.329415: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2599990000 Hz


<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

# Exploring Function Definitions and Graphs

In [7]:
concrete_function.graph

In [9]:
ops = concrete_function.graph.get_operations()

In [11]:
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [12]:
pow_op = ops[2]

In [16]:
pow_op.inputs # A tuple

(<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>)

In [17]:
list(pow_op.inputs) # Convert to a list

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [18]:
pow_op.outputs # Originally a list

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

Fetch operations and tensors by names

In [19]:
concrete_function.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [20]:
concrete_function.graph.get_tensor_by_name('Identity:0')

<tf.Tensor 'Identity:0' shape=() dtype=float32>

Function definition

In [21]:
concrete_function.function_def.signature

name: "__inference_tf_cube_8"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

# A Closer Look at Tracing

In [32]:
@tf.function
def tf_cube(x):
    print("x = ", x)
    return x**3

In [33]:
result = tf_cube(tf.constant(2.0))

x =  Tensor("x:0", shape=(), dtype=float32)


In [34]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [35]:
result = tf_cube(tf.constant(3.0))

In [36]:
result = tf_cube(tf.constant(4.0))

In [37]:
result = tf_cube(2) # new Python value: trace!

x =  2


In [38]:
result = tf_cube(3) # new Python value: trace!

x =  3


In [39]:
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!

x =  Tensor("x:0", shape=(1, 2), dtype=float32)


In [40]:
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!

x =  Tensor("x:0", shape=(2, 2), dtype=float32)


In [41]:
result = tf_cube(tf.constant([[7., 8.], [9., 10.]])) # Same shape: no trace

Manually specify the input signature

In [43]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    return images[:, ::2, ::2] # drop half the rows and columns

In [45]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])
preprocessed_images = shrink(img_batch_1) # Works fine. Trace the function.
preprocessed_images = shrink(img_batch_2) # Works fine. Same concrete function.

In [46]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
preprocessed_images = shrink(img_batch_3) # ValueError! Unexpected signature.

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.09796917 0.20442224]
  [0.913916   0.29718673]]

 [[0.66289246 0.91411257]
  [0.24866903 0.44420063]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None))

In [53]:
@tf.function
def add_10(x):
    for i in range(10):
        x += 1
    return x

In [54]:
add_10(tf.constant(0))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [55]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

In [52]:
@tf.function
def add_10_tf_range(x):
    for i in tf.range(10):
        x += 1
    return x

In [56]:
add_10_tf_range(tf.constant(0))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [57]:
add_10_tf_range.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/limit' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'floordiv' type=FloorDiv>,
 <tf.Operation 'mod' type=FloorMod>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'NotEqual' type=NotEqual>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'zeros_like_1' type=Const>,
 <tf.Operation 'Maximum' type=Maximum>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

# Handling Variablesand Other Resources in TF Functions

In [59]:
counter = tf.Variable(0)

@tf.function
def increment(counter, c=1):
    return counter.assign_add(c)

In [60]:
increment(counter) # counter is now equal to 1
print(counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>


In [61]:
increment(counter) # counter is now equal to 2
print(counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>


In [62]:
function_def = increment.get_concrete_function(counter).function_def

In [63]:
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

Outside resource, not passed as an argument.

In [64]:
counter = tf.Variable(0)

@tf.function
def increment_not_arg(c=1):
    return counter.assign_add(c)

In [65]:
increment_not_arg()
print(counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>


In [66]:
increment_not_arg()
print(counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>


In [67]:
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0)
        
    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)

In [69]:
counter = Counter()
counter.increment()
print(counter.counter)
counter.increment()
print(counter.counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>


# Using TF Functions with tf.keras (or Not)

In [70]:
model = MyModel(dynamic=True)

NameError: name 'MyModel' is not defined

Always being dynamic

In [71]:
class MyLayer(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(dynamic=True, **kwargs)

In [72]:
model.compile(loss=my_mse, optimizer="nadam", metrics=[my_mse], run_eagerly=True)

NameError: name 'model' is not defined